# PPM with implicit FCT
Apply FCT to PPM with a large time step. The first application of FCT should use an implicit upwind method for the bounded solution. FCT then creates a bounded correction of PPM. This can be used as the bounded solution to apply FCT again to the PPM solution. Will this process converge to a more accurate bounded solution?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import matplotlib
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
# Functions for the numerical methods assuming a one-dimensional, uniform, periodic grid
# A periodic domain is implimented using numpy roll.
# Interface j+1/2 is indexed j so that cell j is between interfaces indexed j-1 and j
# Assumes positive velocity so cell j is upwind of interface j-1
#         |  cell |   
#   j-1   |   j   |   j+1
#        j-1      j

def PPMflux(phi, c,options=None):
    """Returns the PPM fluxes for cell values phi for Courant number c.
    Face j is at j+1/2 between cells j and j+1"""
    # Integer and remainder parts of the Courant number
    cI = int(c)
    cR = c - cI
    # phi interpolated onto faces
    phiI = 1/12*(-np.roll(phi,1) + 7*phi + 7*np.roll(phi,-1) - np.roll(phi,-2))
    # Move face interpolants to the departure faces
    if cI > 0:
        phiI = np.roll(phiI,cI)
    # Interface fluxes
    F = np.zeros_like(phi)
    # Contribution to the fluxes from full cells between the face and the departure point
    nx = len(F)
    for j in range(nx):
        for i in range(j-cI+1,j+1):
            F[j] += phi[i%nx]/c
        #F[j] += sum(phi[j-cI+1:j+1])
    # Ratio of remnamt to total Courant number
    cS = 1
    if c > 1:
        cS = cR/c
    # Contribution from the departure cell
    F += cS*( (1 - 2*cR + cR**2)*phiI \
             + (3*cR - 2*cR**2)*np.roll(phi,cI) \
             + (-cR + cR**2)*np.roll(phiI,1))
    return F

def upwindFlux(phi, c,options=None):
    """Returns the first-order upwind fluxes for cell values phi and Courant number c
    Implicit or explicit depending on Courant number"""
    if c <= 1:
        return phi
    nx = len(phi)
    # Off centering for Implicit-Explicit
    a = 1 - 1/c
    # Matrix for implicit solution of
    # phi_j^{n+1} = phi_j^n - (1-a)*c(phi_j^n - phi_{j-1}^n)
    #                       - a*c*(phi_j^{n+1} - phi_{j-1}^{n+1}
    M = diags([-a*c*np.ones(nx-1),  # The diagonal for j-1
               (1+a*c)*np.ones(nx), # The diagonal for j
               [-a*c]], # The top right corner for j-1
              [-1,0,nx-1], # the locations of each of the diagonals
               shape=(nx,nx), format = 'csr')
    # Solve the implicit problem
    phiNew = spsolve(M, phi - (1-a)*c*(phi - np.roll(phi,1)))
    # Back-substitute to get the implicit fluxes
    return (1-a)*phi + a*phiNew

def FCT(phi, c, options={"HO":PPMflux, "LO":upwindFlux, "nCorr":1, 
                         "minPhi": None, "maxPhi": None}):
    """Returns the corrected high-order fluxes with nCorr corrections"""
    # Sort out options
    if not isinstance(options, dict):
        options = {}
    HO =  options["HO"] if "HO" in options else PPMflux
    LO =  options["LO"] if "LO" in options else upwindFlux
    nCorr = options["nCorr"] if "nCorr" in options else 1
    minPhi = options["minPhi"] if "minPhi" in options else None
    maxPhi = options["maxPhi"] if "maxPhi" in options else None
    # First approximation of the bounded flux and the full HO flux
    fluxB = LO(phi,c, options=options)
    fluxH = HO(phi,c, options=options)

    # Add a corrected HO flux
    for it in range(nCorr):
        # The bounded solution
        phid = advect(phi, c, fluxB)
    
        # The allowable min and max
        phiMax = maxPhi
        if phiMax is None:
            phiMax = phid
            if c <= 1:
                phiMax = np.maximum(phi, phiMax)
            phiMax = np.maximum(np.roll(phiMax,1), np.maximum(phiMax, np.roll(phiMax,-1)))
        
        phiMin = minPhi
        if phiMin is None:
            phiMin = phid
            if c <= 1:
                phiMin = np.minimum(phi, phiMin)
            phiMin = np.minimum(np.roll(phiMin,1), np.minimum(phiMin, np.roll(phiMin,-1)))

        # The antidiffusive fluxes
        A = fluxH - fluxB

        # Sums of influxes ad outfluxes
        Pp = c*(np.maximum(0, np.roll(A,1)) - np.minimum(0, A))
        Pm = c*(np.maximum(0, A) - np.minimum(0, np.roll(A,1)))

        # The allowable rise and fall
        Qp = phiMax - phid
        Qm = phid - phiMin

        # Ratios of allowable to HO fluxes
        Rp = np.where(Pp > 1e-12, np.minimum(1, Qp/np.maximum(Pp,1e-12)), 0)
        Rm = np.where(Pm > 1e-12, np.minimum(1, Qm/np.maximum(Pm,1e-12)), 0)

        # The flux limiter
        C = np.where(A >= 0, np.minimum(np.roll(Rp,-1), Rm),
                             np.minimum(Rp, np.roll(Rm,-1)))
        fluxB = fluxB + C*A
        
    return fluxB

def advect(phi, c, flux,options=None):
    """Advect cell values phi using Courant number c using flux"""
    F = flux
    if callable(flux):
        F = flux(phi, c, options=options)
    return phi - c*(F - np.roll(F,1))

In [ ]:
# Initial condition functions

def combi(x):
    "Initial conditions consisting of a square wave and a bell"
    return np.where((x > 0) & (x < 0.4), 0.5*(1-np.cos(2*np.pi*x/.4)),
                    np.where((x > 0.5) & (x < 0.7), 1., 0.))

def square(x):
    "Initial conditions consisting of a square wave"
    return np.where((x > 0) & (x < 0.4), 1., 0.)

def cosBell(x):
    "Initial conditions consisting of a  bell"
    return np.where((x > 0) & (x < 0.4), 0.5*(1-np.cos(2*np.pi*x/.4)), 0)


In [ ]:
# Solutions
# Parameters to compare schemes
nt = 4
nx = 30
dt = 1/12
dx = 1/nx
c = dt*nx
initial = square
x = np.arange(0, 1, dx)
phi0 = initial(x)
plt.plot(x, phi0, 'k', label = 't=0')

nCorrs = 9
fluxes = [PPMflux, upwindFlux] + [FCT] * nCorrs
names = ['PPM', 'upwind']
options = [{}, {}]
for i in range(nCorrs):
    names.append('PPM with FCT '+str(i+1))
    options.append({"nCorr": i+1})

for minPhi in [None, 0]:
    for flux, name, option in zip(fluxes, names, options):
        phi = phi0.copy()
        for it in range(nt):
            phi = advect(phi, c, flux, options=option)
        plt.plot(x, phi, label=name)
    
    plt.legend(bbox_to_anchor=(1.1, 1))
    plt.xlim([0,1])
    if minPhi is None:
        plt.title('Monotonic FCT iterations\n'
              + 'c = '+str(round(c,2))+', dt = '+str(round(dt,2))
               +', nt = '+str(nt)+', nx = '+str(nx))
        plt.savefig('plots/PPM_FCT.pdf')
    else:
        plt.title('Positive FCT iterations\n'
              + 'c = '+str(round(c,2))+', dt = '+str(round(dt,2))
               +', nt = '+str(nt)+', nx = '+str(nx))
        plt.savefig('plots/PPM_FCT+.pdf')
    plt.show()